In [2]:
#########################################################################
## CHUNK PARA CARGA DE VARIABLES COMUNES PARA CUALQUIER TRANSFORMACION ##
#########################################################################
from typing import Any
from data_transformers import chain
from dotenv import load_dotenv
import os
import pandas as pd
from analysislib import (
    get_geonomenclador, 
    exportar_definitivo,
    exportar_transformador,
    compare_ids,
    cargar_verificaciones,
    get_geoerrores,
    gsheet_download_xlsx,
    verificaciones_datasetx,
    match_relocate,
)

from data_transformers.default_transformers import (
    drop_col,
    wide_to_long,
    replace_value,
    rename_cols,
    sort_values,
    cast_col
)

geonomenclador = get_geonomenclador()

# ARGENDATA_WORK_ID = os.getenv("ARGENDATA_WORK_ID")
# gsheet_download_xlsx(id= ARGENDATA_WORK_ID, target="work.xlsx")

work = pd.read_excel('./work.xlsx', sheet_name='Suma de gráficos')
tipo = work[['ID', 'Unnamed: 7', 'Bajada','Publicación']]

In [3]:
##########################################################
## CHUNK PARA CARGA DE VARIABLES COMUNES PARA SUBTOPICO ##
##########################################################

SUBTOP = 'SALING'
entrega = 1
alias = f"{SUBTOP}{entrega}"
folder = f'../output/{alias}'
verificaciones = cargar_verificaciones(alias=alias)

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)

# Cargo funcion para traer datos de dataset para este subtopico- 
verificar_dataset = verificaciones_datasetx(verificaciones=verificaciones)

In [13]:
########################################
## CHUNK PARA CARGA DE DATASET POR ID ##
########################################


grafico_n = 1
archivo = f'{SUBTOP}_g{grafico_n:02d}'

dataset_name = mapping[archivo]
verificaciones_dataset = verificar_dataset(dataset_name=dataset_name)

if verificaciones_dataset:

    geoerrores = get_geoerrores(verificaciones_dataset)

    for col, errores in geoerrores.items():
        for (_, valor, es_correcto) in errores:
            if not es_correcto:
                print(f"No se encontró '{valor}' en la columna {col}")

df = pd.read_csv(f'{folder}/definitivos/{archivo}.csv')

exportar = (lambda df:
                exportar_definitivo(archivo=archivo,
                                    folder=folder,
                                    df=df,
                                    nuevo_nombre=None))

fila_work = tipo[tipo.ID.map(compare_ids(archivo))]

if len(fila_work)>0:

    _, tipologia_target, bajada, va = tipo[tipo.ID.map(compare_ids(archivo))].iloc[0]

    va = va.lower() != 'no va'

    print(f"Va? = {'Sí' if va else 'No'}")
    print(f"Tipología target = '{tipologia_target}'")
    print(f"Bajada = '{bajada}'")

else:
    print(f"El gráfico {archivo} no se encuentra en la plantilla 'WORK'")


plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print('Primary keys:', ', '.join(f'"{x}"' for x in primary_keys))
df.head()

No tiene geocontroles
El gráfico SALING_g01 no se encuentra en la plantilla 'WORK'
Primary keys: "year", "semestre"


,year,semestre,indicador,valor
0,2003,2,ipcf_index,100.00000
1,2004,1,ipcf_index,102.31232
2,2004,2,ipcf_index,107.82178
3,2005,1,ipcf_index,109.70754
4,2005,2,ipcf_index,118.60464


In [4]:
#######################################################
## CHUNK PARA ARMADO DE PIPELINE DE TRANSFORMACIONES ##
#######################################################


pipeline = chain(
    rename_cols({'sector': 'indicador'})
)

callstack, result = pipeline(df)

result.head()

,indicador,valor
0,Alimentos y bebidas no alcoholicas,26.93381
1,Bebidas alcoholicas y tabaco,3.49540
2,Prendas de vestir y calzado,9.90447
3,Vivienda agua electricidad gas y otros combust...,9.43438
4,Equipamiento y mantenimiento del hogar,6.36303


In [5]:
#########################################################
## CHUNK PARA EXPORTAR PIPELINE COMO ARCHIVO DE PYTHON ##
#########################################################

exportar_transformador(archivo, pipeline, callstack)

'PRECIO_g01_transformer.py'

In [12]:
#######################################################################
## CHUNK PARA EXPORTAR DATASET COMO CSV Y RENOMBRAR DATASET ORIGINAL ##
#######################################################################

exportar(result)

('../output/PRECIO1/definitivos/PRECIO_g01.csv',
 '../output/PRECIO1/definitivos/PRECIO_g01_old.csv')

In [ ]:
#####################################################################
## CHUNK PARA CREAR CARPETA OLD Y PASAR AHI TODOS LOS ARCHIVOS OLD ##
#####################################################################

# match_relocate(f'{folder}/definitivos', f'{folder}/old', '.*old.*')